In [ ]:
import pprint
import requests
import time

import numpy as np
import torch
import torchvision

import matplotlib.pyplot as plt
import torch.nn.functional as F

from io import BytesIO
from PIL import Image

# Lakota AI Code Camp Lesson 07: Matrix Algebra III

We're going to talk about
*   matrices;
*   matrix multiplication;
*   gaxpy.

## Matrices

An important concept in mathematics is something called a **linear transformation**.
It is a function that takes a vector of dimension $n$ and gives a vector of dimension $m$.
An important property is that a linear transformation respects the vector operations of vector addition and scalar multiplication.

In mathematical terms, if $T$ is a linear transformation and $\textbf{v}$ and $\textbf{w}$ are vectors of dimension $n$, then we must have
$$
T(\textbf{v} + \textbf{w}) = T(\textbf{v}) + T(\textbf{w}).
$$

You have seen something similar with distributivity.
$$
a \cdot (b + c) = a\cdot b + a \cdot c,
$$
where $a, b$, and $c$ are numbers.

Another property we require is that a linear transformation respects scalar multiplication.
So, if $c$ is a scalar and $\textbf{v}$ is a vector, then
$$
T(c \textbf{v}) = c T(\textbf{v}).
$$

An amazing property of linear transformations, that you will learn in a full linear algebra course, is that every linear transformation can be represented by a matrix.

Let's give an example:
let $\textbf{v} = (v_{1}, v_{2}, v_{3})$, then let
$$
T(\textbf{v}) = (2v_{1} + v_{3}, - v_{1} + 5v_{2}) = (2v_{1} + 0v_{2} + v_{3}, - v_{1} + 5v_{2} + 0v_{3}).
$$
So, the matrix representing this is:
$$
\left(
\begin{array}
& 2 & 0 & 1 \\
-1 & 5 & 0 \\
\end{array}
\right)
$$

Another example of a matrix is
$$
\left(
\begin{array}
& 1 & 2 & 3 \\
4 & 5 & 6 \\
\end{array}
\right)
$$
and
$$
\left(
\begin{array}
& 1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1 \\
\end{array}
\right)
$$

We say a matrix is of size $n \times m$ if there are $n$ rows and $m$ columns.

Another way to write a matrix is
$$
\textbf{A} = (a_{ij})_{ij}.
$$
This tells us the $i$th row and the $j$th column is the value $a_{ij}$.
It can be a useful method of representing a matrix abstractly.

Another thing to know about a matrix is that every matrix has a transpose.
It will be useful shortly.
If
$$
\textbf{A} = (a_{ij})_{ij}.
$$
then the transpose of $\textbf{A}$ is $\textbf{A}^{\intercal}$ and
$$
\textbf{A}^{\intercal} = (a_{ij})_{ij}.
$$
The columns become the rows and the rows become the columns.
Another way to think about it is that the matrix is reflected about the diagonal.

### Matrix Operations

A matrix has similar operations to a vector, but a few more:

*   the number of components cannot change, i.e. the number of rows and columns are fixed;
*   we need to be able to add two matrices together;
*   we need to be able to multiply a matrix by a real number (recall, that in this context, we call it a scalar);
*   we need to be able to get the individual components.

There are several nice-to-haves, that wouldn't typically affect our functionality, but are tremendous quality-of-life improvements:

*   we would like to have a nice way to print the number;
*   we would like to get the size of the matrix (i.e. the number of rows and columns).

Let's work on making a matrix class!

In [ ]:
class Vector():

    def __init__(self, values):
        # This should have an input.
        # What do we need to input to create a vector?
        self.values = tuple(values)

    def __getitem__(self, idx):
        return self.values[idx]

    def __len__(self):

        length = 0
        for val in self.values:
            length += 1
        return length

    def __add__(self, other):
        if self.__len__() != other.__len__():
            raise Exception(f"Dimension mismatch: {self.__len__()} != {other.__len__}")

        add = []
        for x, y in zip(self.values, other.values):
            add.append(x + y)
        return Vector(add)

    def __sub__(self, other):
        if self.__len__() != other.__len__():
            raise Exception(f"Dimension mismatch: {self.__len__()} != {other.__len__}")

        sub = []
        for x, y in zip(self.values, other.values):
            sub.append(x - y)
        return Vector(sub)

    def __mul__(self, scalar):
        if type(scalar) not in [int, float]:
            raise Exception(f"{scalar} is not an integer or a float")

        mul = []
        for val in self.values:
            mul.append(val * scalar)
        return Vector(mul)

    def __repr__(self):
        return f"Vector({self.values})"

    def __str__(self):
        vec_string = ''
        for idx in range(self.__len__()):
            vec_string = vec_string + str(self[idx]) + ', '
        vec_string = "(" + vec_string[:-2] + ")"
        return "Vector" + vec_string

In [ ]:
class Matrix():

    def __init__(self, values):
        r"""
        Args:
            values (list of lists): A list of n (rows) lists.
                The n lists all have the same length m (columns).
        """
        self.rows = len(values)
        self.cols = len(values[0])
        self.shape = (self.rows, self.cols)
        self.values = self._create_matrix(values)

    def _create_matrix(self, values):

        for num in range(self.rows):
            if len(values[num]) != self.cols:
                raise Exception(f"Dimension mismatch: {len(num)} != {self.cols}")

        for num in range(self.rows):
            values[num] = list(values[num])

        return list(values)

    def __repr__(self):
        # This needs to return a string that tells you the class and some values.
        # Another way to think about it is that you need to give someone the right amount of
        # information to understand this class.
        return f"Matrix({self.values})"

    def __getitem__(self, i, j=None):
        # i is the row you want to access and j is the column you want to access.
        # Reference the vector class above.
        return self.values[i][j] if j else self.values[i]

    def __len__(self):
        # This should return an integer.
        return self.rows

    def __str__(self):
        # This should return a string that makes the elements easy to read.
        matrix_string = '['
        for rows in range(self.rows):
            matrix_string += '['
            for cols in range(self.cols):
                matrix_string += str(self.values[rows][cols]) + ', '
            matrix_string = matrix_string[:-2] + ']\n '
        matrix_string = matrix_string[:-2] + ']'
        return matrix_string

    def transpose(self):
        # Initialize
        transpose = [[0 for i in range(self.rows)] for i in range(self.cols)]

        # Set the values
        for row in range(self.rows):
            for col in range(self.cols):
                transpose[col][row] = self.values[row][col]

        return Matrix(transpose)

    def __add__(self, other):
        if not(self.rows == other.rows and self.cols == other.cols):
            raise Exception("The rows or columns do not match.")

        add = [[None for i in range(self.cols)] for j in range(self.rows)]

        for row in range(self.rows):
            for col in range(self.cols):
                add[row][col] = self.values[row][col] + other.values[row][col]

        return Matrix(add)

    def __sub__(self, other):
        if not(self.rows == other.rows and self.cols == other.cols):
            raise Exception("The rows or columns do not match.")

        sub = [[None for i in range(self.cols)] for j in range(self.rows)]

        for row in range(self.rows):
            for col in range(self.cols):
                sub[row][col] = self.values[row][col] - other.values[row][col]

        return Matrix(sub)

In [ ]:
x = Matrix([[1, 2, 3], [4, 5, 6]])
print(x)

[[1, 2, 3]
 [4, 5, 6]]


In [ ]:
y = Matrix([[2, 4, 6], [8, 10, 12]])

print(x + y)

[[3, 6, 9]
 [12, 15, 18]]


In [ ]:
x[0]

[1, 2, 3]

In [ ]:
x[0][2]

3

In [ ]:
x.transpose()

Matrix([[1, 4], [2, 5], [3, 6]])

### Matrix Vector Multiplication

If our input is an $n \times m$ matrix $\textbf{A} = (a_{ij})_{ij}$ and an $m$-dimensional vector $\textbf{x} = (x_{1}, \ldots, x_{n})$, then matrix-vector multiplication takes a matrix and a vector sas input and outputs a vector.

In [ ]:
def dot_product(vector1, vector2):
    # We need to initialize a value
    dot = 0

    # We need to do a for loop
    for num in range(len(vector1)):
        dot += vector1[num] * vector2[num]

    return dot

In [ ]:
def mat_vec_mul(matrix, vector):
    # Our input is a matrix and a vector
    # Our output is a vector
    out = [0 for i in range(matrix.rows)]

    for row in range(matrix.rows):
        out[row] = dot_product(Vector(matrix[row]), vector)

#        for col in range(matrix.cols):
#            out[row] += matrix[row][col] * vector[col]

    return Vector(out)

In [ ]:
x = Matrix([[1, 2, 3], [4, 5, 6]])
y = Vector([1, 2, 3])


# [14, 32]
mat_vec_mul(x, y)

Vector((14, 32))